## Your Model 🌱Garden🌱 Execution Environment

Use this notebook to write a function that executes your model(s). Tag that function with the `@garden_entrypoint` decorator.

Garden will take this notebook and build a container with it. When Garden executes your `@garden_entrypoint`, it will be like like you have just run all the cells of this notebook once. So you can install libraries with `!pip install` and your function can use those libraries. You can also define helper functions and constants to use in your `@garden_entrypoint`.

In [ ]:
from garden_ai.model_connectors import create_connector
from garden_ai import EntrypointMetadata, garden_entrypoint, entrypoint_test

In [ ]:
import torch

### Model connectors

Model connectors let Garden import metadata about your model.
They also have a `stage` method that you can use to download your model weights.

In [ ]:
my_hugging_face_repo = create_connector("https://huggingface.co/Garden-AI/sample_sklearn_model")

### Entrypoint metadata


To publish your function, Garden needs metadata so that other users can discover it.
Edit this EntrypointMetadata object to describe your function.


In [ ]:
my_entrypoint_meta = EntrypointMetadata(
    title="My Inference Function",
    description="Write a longer description here so that people know what your entrypoint does.",
    authors=["you", "your collaborator"],
    tags=["materials science", "your actual field"]
)

### Helper Functions

Define any helper functions you need and use them in the function you want to let people run remotely

In [ ]:
def preprocess(input_tensor):
    min_val, max_val = 0, 1
    min_tensor = torch.min(input_tensor)
    max_tensor = torch.max(input_tensor)
    scaled_tensor = (input_tensor - min_tensor) / (max_tensor - min_tensor)
    scaled_tensor = scaled_tensor * (max_val - min_val) + min_val
    return scaled_tensor

### Write your entrypoint function that will run remotely

The `@garden_entrypoint` decorator makes this function available to run in your garden when you publish the notebook.
Download your model weights and call your model in this function.

In the decorator be sure to include:
- your entrypoint metadata,
- connectors for any models you're using,

You can add your entrypoint to a Garden in two different ways.

If you want all entrypoints in this notebook be added to one Garden, set the `NOTEBOOK_GLOBAL_DOI` in your notebooks metadata or by using the `--doi` argument for `garden-ai notebook start`

If you want to specify different Gardens for different entrypoints, provide each decorator with the optional `garden_doi` argument.

If you both set the `NOTEBOOK_GLOBAL_DOI` and are providing a decorator with a DOI, the entrypoint will ONLY be added to the Garden given to the decorator.

To see all the DOIs of your gardens, use `garden-ai garden list`

In [ ]:
@garden_entrypoint(metadata=my_entrypoint_meta,  model_connectors=[my_hugging_face_repo], garden_doi="10.23677/my-garden-doi")
def run_my_model(input_tensor):
    download_path = my_hugging_face_repo.stage()
    from model_definition_module_in_the_hf_repo import MyModel
    model = MyModel()
    model.load_state_dict(torch.load(download_path + "/my_model.pt"))
    model.eval()

    scaled_tensor = preprocess(input_tensor)
    with torch.no_grad():
        output = model(scaled_tensor)
    
    return output

### Test your entrypoint function

Finally, make sure your `@garden_entrypoint` works!
When Garden makes a container from your notebook, it runs all the cells in order and saves the notebook. Then users invoke your `@garden_entrypoint` in the context of the notebook.

Note on testing: any test functions that call your entrypoint (like the one below) should be marked with `@entrypoint_test(<entrypoint_being_tested>)`. This is because calling an entrypoint typically causes side-effects (such as downloading your model weights to disk) that shouldn't be "baked in" to the environment of the final published notebook. 

Anything marked with `@entrypoint_test` won't be run at publication time, so you don't need to remember to comment out your test code before publishing. We'll also use `@entrypoint_test` functions as example code for others to see how your entrypoint expects to be called. 

In [ ]:
@entrypoint_test(run_my_model)
def test_run_my_model():
    # Replace with input that is relevant for your garden_entrypoint
    example_input = torch.tensor([
        [1, -1], [3, 2]
    ])
    return run_my_model(example_input)
    
test_run_my_model()